# SCC-ICMC-USP 


# CURSO 2 - CD-AM-DM
# Exemplo 8 - SVM (SUPPORT VECTOR MACHINE), NB (NAIVE-BAYES) E KNN (K-NEAREST NEIGHBOUR) 
# Profa. Roseli A. F. Romero

---

### Objetivos:  COMPARAÇÃO DE DESEMPENHO DE DIFERENTES MÉTODOS DE AM
- USO DE 3 IMPORTANTES CLASSIFICADORES: SVM, NB E KNN
- USO DE DIFERENTES PARAMETRIZAÇÕES PARA CADA MÉTODO
- USO DA CURVA ROC 

---

### Questão 1.

Carregue o dataset Breast Cancer, faça uma análise de dados faltantes e separe o dataset em 75% treino e 25% teste:

In [ ]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

# Carregando dados do sklearn:
dataset_sklearn = load_breast_cancer()
# Criando lista com nome das colunas:
nome_colunas = list(dataset_sklearn['feature_names'])+ ['target']
# Concatenando dados + alvo:
dados = np.c_[dataset_sklearn['data'], dataset_sklearn['target']]
# Criando dataframe:
dataset = pd.DataFrame(data=dados, columns=nome_colunas)
dataset

- Exploração inicial do conjunto:

In [ ]:
dataset.info()

- Verificando valores faltantes:

In [ ]:
dataset.isnull().sum()

- Separando o conjunto:

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(dataset, test_size=0.25, random_state=0)

In [ ]:
train_data

In [ ]:
test_data

### Questão 02.

Prepare os dados para treino, padronizando e dividindo-os em atributos e classes. Crie também uma estrutura do tipo dicionário que armazene nome de modelos, acurácias, recall e área sob curva roc.

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train, y_train = train_data.drop(['target'], axis=1), train_data['target']
X_test, y_test = test_data.drop(['target'], axis=1), test_data['target']

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
# Criando a estrutura do tipo dicionário
resultados = {
    "Algoritmo" :[],
    "Parametrização" :[],
    "Acuracia" :[],
    "Recall" :[],
    "Roc_auc" :[]
}

- Para facilitar, vamos criar uma função que recebe o tipo do algoritmo, um conjunto de parametrizações, um conjunto de treino, um conjunto de teste e uma estrutura para armazenar métricas de acurácia, recall e roc auc.

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

def avaliar_modelos(tipo, parametrizacoes, X_treino, y_treino, X_teste, y_teste, resultados):
    for parametrizacao in parametrizacoes:
        print("Avaliando", tipo, "com parametrização:", parametrizacao)
        classificador = parametrizacoes[parametrizacao]
        classificador.fit(X_treino, y_treino)
        y_pred = classificador.predict(X_teste)        
        y_score = classificador.predict_proba(X_teste)[:, 1]
        # Adicionando aos resultados:
        resultados["Algoritmo"].append(tipo)
        resultados["Parametrização"].append(parametrizacao)
        resultados["Acuracia"].append(accuracy_score(y_test, y_pred))
        resultados["Recall"].append(recall_score(y_test, y_pred))
        resultados["Roc_auc"].append(roc_auc_score(y_test, y_score))

### Questão 03.

Classifique o conjunto de treino utilizando o algoritmo SVM. Treine ao menos 5 configurações diferentes.

In [ ]:
from sklearn.svm import SVC
# definindo as diferentes configurações para o método SVM
modelos_svm = {
    "Kernel rbf, gamma auto": SVC(kernel = 'rbf', gamma='auto', probability=True),
    "Kernel linear": SVC(kernel = 'linear', probability=True),
    "Kernel sigmoide": SVC(kernel = 'sigmoid', probability=True),
    "Kernel polinomial grau 2": SVC(kernel = 'poly', degree=2, probability=True),
    "Kernel polinomial grau 3": SVC(kernel = 'poly', degree=3, probability=True)
}

avaliar_modelos("SVM", modelos_svm, X_train, y_train, X_test, y_test, resultados)

### Questão 04.

Classifique o conjunto de treino utilizando o algoritmo Gaussian Naive Bayes. Treine ao menos 5 configurações diferentes.

In [ ]:
from sklearn.naive_bayes import GaussianNB
# definindo as diferentes configurações para o método NB
modelos_gaussian = {
    "Var smoothing 1e-9": GaussianNB(var_smoothing=1e-9),
    "Var smoothing 1e-8": GaussianNB(var_smoothing=1e-8),
    "Var smoothing 1e-7": GaussianNB(var_smoothing=1e-7),
    "Var smoothing 1e-6": GaussianNB(var_smoothing=1e-6),
    "Var smoothing 1e-5": GaussianNB(var_smoothing=1e-5)
}

avaliar_modelos("GaussianNB", modelos_gaussian, X_train, y_train, X_test, y_test, resultados)

### Questão 05.

Por fim, classifique o conjunto de treino utilizando o algoritmo KNN. Treine ao menos 5 configurações diferentes.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# definindo as diferentes configurações para o método KNN
modelos_knn = {
    "N=3": KNeighborsClassifier(n_neighbors=3),
    "N=5": KNeighborsClassifier(n_neighbors=5),
    "N=7": KNeighborsClassifier(n_neighbors=7),
    "N=9": KNeighborsClassifier(n_neighbors=9),
    "N=11": KNeighborsClassifier(n_neighbors=11),
}

avaliar_modelos("KNN", modelos_knn, X_train, y_train, X_test, y_test, resultados)

### Questão 06.

Determine a melhor parametrização para cada tipo de classificador (Gaussian Naive Bayes, KNN e SVM) considerando a métrica Roc Auc. Depois plote a curva Roc dessas três melhores parametrizações.

- Vamos gerar um dataframe com o dicionário de resultados para podermos manipulá-lo mais fácilmente:

In [ ]:
resultados_df = pd.DataFrame.from_dict(resultados)
resultados_df

- Queremos achar as parametrizações que geraram maior roc auc entre os algoritmos testados. Para isso, podemos usar a função groupby e aggregate do pandas:

In [ ]:
best_roc_aucs = resultados_df.groupby(["Algoritmo"])["Roc_auc"].agg([ ("Roc_auc", max) ])
best_roc_aucs

- Agora vamos recuperar os modelos com melhores parametrizações:

In [ ]:
melhores_parametrizacoes = {}
for linha in best_roc_aucs.itertuples():
    tipo_algo = linha[0]
    melhor_valor = linha[1]    
    # Colocamos iloc[0] no final para pegar a primeira ocorrencia, pois podemos
    # ter mais de uma mesma parametrização com mesmo valor roc_aoc:
    melhores_parametrizacoes[tipo_algo] = resultados_df.query(
        "(Algoritmo==@tipo_algo) & (Roc_auc==@melhor_valor)").iloc[0]["Parametrização"]
    
    print("Melhor parametrização do", tipo_algo, "=", melhores_parametrizacoes[tipo_algo])

melhor_gaussiannb = modelos_gaussian[melhores_parametrizacoes["GaussianNB"]]
melhor_KNN = modelos_knn[melhores_parametrizacoes["KNN"]]
melhor_svm = modelos_svm[melhores_parametrizacoes["SVM"]]

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

plt.figure(figsize=(12,12))
lw = 2

y_score = melhor_gaussiannb.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=1)
roc_auc = best_roc_aucs.loc["GaussianNB"]
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='Gaussian (area = %0.4f)' % roc_auc)

y_score = melhor_KNN.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=1)
roc_auc = best_roc_aucs.loc["KNN"]
plt.plot(fpr, tpr, color='darkgreen', lw=lw, label='KNN (area = %0.4f)' % roc_auc)

y_score = melhor_svm.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=1)
roc_auc = best_roc_aucs.loc["SVM"]
plt.plot(fpr, tpr, color='darkblue', lw=lw, label='SVM (area = %0.4f)' % roc_auc)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa Falsos Positivos')
plt.ylabel('Taxa Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")

Portanto, segundo a métrica de ROC Auc, o melhor modelo é o SVM com kernel=rbf e gamma = auto